<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第13章『経時データ・パネルデータ解析入門』" data-toc-modified-id="第13章『経時データ・パネルデータ解析入門』-1">第13章『経時データ・パネルデータ解析入門』</a></span><ul class="toc-item"><li><span><a href="#p.6~7" data-toc-modified-id="p.6~7-1.1">p.6~7</a></span></li><li><span><a href="#p.8~10" data-toc-modified-id="p.8~10-1.2">p.8~10</a></span></li><li><span><a href="#p.14~15" data-toc-modified-id="p.14~15-1.3">p.14~15</a></span></li><li><span><a href="#p.17~19" data-toc-modified-id="p.17~19-1.4">p.17~19</a></span></li><li><span><a href="#p.20~21" data-toc-modified-id="p.20~21-1.5">p.20~21</a></span></li></ul></li></ul></div>

# 第13章『経時データ・パネルデータ解析入門』

## p.6~7

In [1]:
dat <- read.csv("datPR1964.csv") 

datR <- dat[dat$sex=="F",]
datR$id <- factor(datR$id)
datR$age <- factor(datR$age) 
n<-length(levels(datR$id))
p <- length(levels(datR$age))
bary <- mean(datR$dis)
baryp <- rep(by(datR$dis, datR$age, mean), n)
baryn <- rep(by(datR$dis, datR$id, mean), rep(4, n))
s2b <- sum((baryp- bary)^2) 
s2d<-sum((baryn-bary)^2)
s2e <- sum((datR$dis - baryp - baryn + bary)^2)
s2t <- (n*p-1)*var(datR$dis)
tils2b <- s2b/(p-1)
tils2d <- s2d/(n-1)
tils2e <- s2e/((p-1)*(n-1))

c(p-1, s2b, tils2b, tils2b/tils2e)
c(n-1, s2d, tils2d, tils2d/tils2e)
c((p-1)*(n-1), s2e, tils2e)
c(n*p-1, s2t)

1-pf(tils2b/tils2e, p-1, (p-1)*(n-1))
1-pf(tils2d/tils2e, n-1, (p-1)*(n-1))

## p.8~10

In [2]:
dat <- read.csv("datPR1964.csv") 

dat$id <- factor(dat$id)
dat$age <- factor(dat$age)
dat$sex <- factor(dat$sex)
np <- dim(datR)[1]
n <- length(levels(dat$id)) 
p<-length(levels(dat$age))
q <- length(levels(dat$sex))

bary <- mean(dat$dis)
baryp <- rep(by(dat$dis, dat$age, mean), n)
baryn <- rep(by(dat$dis, dat$id, mean), rep(4, n)) 
baryq<-rep(by(dat$dis,dat$sex,mean),summary(dat$sex))

barypq <- NULL
for(kk in 1:q){
    dattemp <- dat[dat$sex==levels(dat$sex)[kk],]
    barypq <- c(barypq, rep(by(dattemp$dis,dattemp$age,mean),dim(dattemp)[1]/p))
}

s2a <- sum((baryq -bary)^2)
s2b <- sum((baryp -bary)^2)
s2g <- sum((barypq -baryq -baryp +bary)^2)
s2d <- sum((baryn -baryq)^2)
s2e <- sum((dat$dis -barypq -baryn +baryq)^2)
s2t <- (n*p-1)*var(dat$dis) 
tils2a<-s2a/(q-1)
tils2b <- s2b/(p-1)
tils2g <- s2g/((q-1)*(p-1))
tils2d <- s2d/(n-q)
tils2e <- s2e/((p-1)*(n-1))

c(q-1, s2a, tils2a, tils2a/tils2d)
c(p-1, s2b, tils2b, tils2b/tils2e)
c((q-1)*(p-1), s2g, tils2g, tils2g/tils2e)
c(n-q, s2d, tils2d, tils2d/tils2e)
c((n-q)*(p-1), s2e, tils2e) c(n*p-1,s2t)

1-pf(tils2a/tils2d, q-1, n-q)
1-pf(tils2b/tils2e, p-1, (n-q)*(p-1))
1-pf(tils2g/tils2e, (q-1)*(p-1), (n-q)*(p-1))
1-pf(tils2d/tils2e, n-q, (n-q)*(p-1))

## p.14~15

In [3]:
dat <- read.csv("datPR1964.csv")

datR<-dat[dat$sex=="F",]
datR$id <- factor(datR$id)
datR$age <- factor(datR$age)
n <- length(levels(datR$id))
p <- length(levels(datR$age))
m <- n-1 
nX<-cbind(rep(1,p),c(-3,-1,1,3))
q <- dim(nX)[2]

daty <- NULL
for(ii in 1:n){
    ind <- datR$id==levels(factor(datR$id))[ii]
    daty <- rbind(daty, datR[ind,]$dis)
}
bary <- apply(daty, 2, mean)
mS <- var(daty)
iS <- solve(mS) XiSX<-t(nX)%*%iS%*%nX
XiSX<-t(nX)%*%iS%*%nX
hvtheta<-solve(XiSX)%*%t(nX)%*%iS%*%bary
hvar <- (m-1)/(m-(p-q)-1) * m/(m-(p-q)) * 1/n * solve(XiSX)
#推定値 
cbind(hvtheta, diag(hvar))

#切片の検定
D <- matrix( c(1, 0), 2, 1)
d <- dim(D)[2]
T2 <- m*t(bary) %*% (iS-iS%*%nX%*%solve(XiSX)%*%t(nX)%*%iS) %*% bary
R <- 1/n+1/m*T2 
Se<-m*t(D)%*%solve(XiSX)%*%D
Sh <- 1/R *(t(D)%*%hvtheta) %*% t(t(D)%*%hvtheta)
Lam <- det(Se)/det(Se+Sh)
TD <- (m-(p-q)-d+1)/d * (1-Lam)/Lam
TD
1-pf(TD, d, m-(p-q)-d+1)

#傾きの検定 
D <- matrix( c(0, 1), 2, 1)
d <- dim(D)[2]
T2 <- m*t(bary) %*% (iS-iS%*%nX%*%solve(XiSX)%*%t(nX)%*%iS) %*% bary 
R<-1/n+1/m*T2
Se <- m*t(D)%*%solve(XiSX)%*%D
Sh <- 1/R *(t(D)%*%hvtheta) %*% t(t(D)%*%hvtheta)
Lam <- det(Se)/det(Se+Sh)
TD <- (m-(p-q)-d+1)/d * (1-Lam)/Lam 
TD
1-pf(TD, d, m-(p-q)-d+1)

## p.17~19

In [4]:
dat <- read.csv("datPR1964.csv")
dat$sex <- factor(dat$sex)
dat$id <- factor(dat$id)
dat$age <- factor(dat$age)

k<-length(levels(dat$sex))
n <- length(levels(dat$id))
p <- length(levels(dat$age))
m <- n-k
nX <- cbind(rep(1, p), c(-3,-1,1,3), c(1,-1,-1,1))
q <- dim(nX)[2]

nis <- NULL
bary <- NULL
mS <- matrix(0,p,p)
for(ii in 1:k){
    daty <- NULL
    datR <- dat[dat$sex==levels(dat$sex)[ii],]
    datR$id <- factor(datR$id)
    ni <- length(levels(datR$id))
    for(jj in 1:ni){
        ind <- datR$id==levels(factor(datR$id))[jj] 
        daty<-rbind(daty,datR[ind,]$dis)
    }
    nis<-c(nis,ni)
    bary <- cbind(bary, apply(daty, 2, mean))
    mS <- mS + (ni-1)*var(daty)
}
mS <- 1/(n-k)*mS iS<-solve(mS)
iS <- solve(mS)
XiSX <- t(nX) %*% iS %*% nX

tempsum <- NULL
hTheta <- NULL
for(ii in 1:k){
    ni<-nis[ii]
    hvtheta <- solve(XiSX) %*% t(nX) %*% iS %*% bary[,ii]
    hvar <- (m-1)/(m-(p-q)-1) * m/(m-(p-q)) * 1/ni * solve(XiSX)
    #推定値 
    tempsum <- rbind(tempsum, cbind(hvtheta, diag(hvar)))
    hTheta <- rbind(hTheta, t(hvtheta))
}
tempsum

TDs <- NULL
Ps <- NULL
for(kk in 1:k){
    vc<-as.matrix(numeric(k))
    vc[kk] <- 1
    for(qq in 1:q){
        D <- as.matrix(numeric(q))
        D[qq] <- 1 
        d<-dim(D)[2]
        T2 <- m*t(bary) %*% (iS-iS%*%nX%*%solve(XiSX)%*%t(nX)%*%iS) %*% bary
        R <- diag(1/nis)+1/m*T2
        Se <- m*t(D)%*%solve(XiSX)%*%D
        Sh <- 1/(t(vc)%*%R%*%vc) *(t(vc)%*%hTheta%*%D) %*% t(t(vc)%*% hTheta%*%D)
        Lam <- det(Se)/det(Se+Sh) 
        TD<-(m-(p-q)-d+1)/d*(1-Lam)/Lam
        TDs <- c(TDs, TD)
        Ps <- c(Ps, 1-pf(TD, d, m-(p-q)-d+1))
    }
}
TDs
Ps

## p.20~21

In [6]:
library(nlme) 
dat<-read.csv("datPR1964.csv")
dat$sex<-factor(dat$sex)
dat$id <- factor(dat$id)

#データの調整 
temp <- groupedData(dis~age|id, data=dat)

#線形混合モデルで分析 
result <- lme(dis~age+sex+age:sex, data=temp, random=~age, method="ML")

summary(result)